In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, os.path

dataset = '/content/drive/Shareddrives/229 proj/compressed_dataset'
# print(os.listdir(dataset))
dict = {}
for country in os.listdir(dataset):
    f_country = dataset+'/'+country
    if os.path.isdir(f_country):
        n_pics = len(os.listdir(f_country))
        if n_pics > 0:
            dict[n_pics] = country
            # print(f"country: {n_pics}. n_pics: {country}")

# for key in sorted(dict.keys()):
    # print(f"#pics: {key} = {dict[key]}")
A = []
for key in sorted(dict.keys()):
    A.append(f"#pics: {key} = {dict[key]}")
for i in range(-1,-len(A),-1):
    print(f"{i}: {A[i]}")

-1: #pics: 12062 = United States
-2: #pics: 3867 = Japan
-3: #pics: 3581 = France
-4: #pics: 2504 = United Kingdom
-5: #pics: 2337 = Brazil
-6: #pics: 1761 = Russia
-7: #pics: 1712 = Australia
-8: #pics: 1382 = Canada
-9: #pics: 1183 = South Africa
-10: #pics: 1075 = Spain
-11: #pics: 1049 = Finland
-12: #pics: 953 = Thailand
-13: #pics: 907 = Mexico
-14: #pics: 872 = Poland
-15: #pics: 789 = Italy
-16: #pics: 726 = Sweden
-17: #pics: 716 = Singapore
-18: #pics: 698 = Germany
-19: #pics: 697 = Argentina
-20: #pics: 675 = Norway
-21: #pics: 588 = Netherlands
-22: #pics: 557 = New Zealand
-23: #pics: 547 = Taiwan
-24: #pics: 423 = Malaysia
-25: #pics: 347 = Austria
-26: #pics: 346 = Romania
-27: #pics: 326 = Chile
-28: #pics: 290 = Ireland
-29: #pics: 288 = Indonesia
-30: #pics: 270 = Peru
-31: #pics: 268 = Turkey
-32: #pics: 264 = Czechia
-33: #pics: 251 = Colombia
-34: #pics: 248 = Greece
-35: #pics: 243 = South Korea
-36: #pics: 242 = Portugal
-37: #pics: 219 = Philippines
-38: #pics:

In [ ]:
# import resnet50 model and weights 
import torch
from torchvision import datasets, models, transforms
from torchvision.models import resnet50, ResNet50_Weights, alexnet, AlexNet_Weights, squeezenet1_0, SqueezeNet1_0_Weights, vgg16, VGG16_Weights, densenet201, DenseNet201_Weights, inception_v3, Inception_V3_Weights
from torchvision.datasets import ImageFolder
import torch.optim as optim
from torch import nn
import time
import copy

In [ ]:

############# Baseline Section ############### 
# Feature extraction (not finetuning). 

# Select the most up-to-date weights (default)
model_resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
model_alexnet = alexnet(weights=AlexNet_Weights.DEFAULT)
model_squeezenet = squeezenet1_0(weights=SqueezeNet1_0_Weights.DEFAULT)
model_vgg = vgg16(weights=VGG16_Weights.DEFAULT)
model_densenet = densenet201(weights=DenseNet201_Weights.DEFAULT)
model_inception = inception_v3(weights=Inception_V3_Weights.DEFAULT)

# Initialize the Weight Transforms
weights_resnet = ResNet50_Weights.DEFAULT
weights_alexnet = AlexNet_Weights.DEFAULT
weights_squeezenet = SqueezeNet1_0_Weights.DEFAULT
weights_vgg = VGG16_Weights.DEFAULT
weights_densenet = DenseNet201_Weights.DEFAULT
weights_inception = DenseNet201_Weights.DEFAULT


preprocess_resnet = weights_resnet.transforms() 
preprocess_alexnet = weights_alexnet.transforms() 
preprocess_squeezenet = weights_squeezenet.transforms() 
preprocess_vgg = weights_vgg.transforms() 
preprocess_densenet = weights_densenet.transforms() 
preprocess_inception = weights_inception.transforms() 
# run the images through this preprocess()
# e.g: batch = preprocess(img).unsqueeze(0)

# data_path = 'drive/Shareddrives/229 proj/compressed_dataset'

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_names = ["resnet", "alexnet", "vgg", "squeezenet", "densenet", "inception"]

# Number of classes in the dataset
num_classes = 124

# Batch size for training (change depending on how much memory you have)
batch_size = 32

# Number of epochs to train for
num_epochs = 16

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_0-b66bff10.pth


...

  0%|          | 0.00/528M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth


  0%|          | 0.00/77.4M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

In [ ]:
# if feature_extracting is on, we don't want to update the weights in the network
# except for the last layer. This freezes function the model params. 
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
set_parameter_requires_grad(model_resnet, feature_extract)
set_parameter_requires_grad(model_alexnet, feature_extract)
set_parameter_requires_grad(model_squeezenet, feature_extract)
set_parameter_requires_grad(model_vgg, feature_extract)
set_parameter_requires_grad(model_densenet, feature_extract)
set_parameter_requires_grad(model_inception, feature_extract)

In [ ]:
# Data and model initialization 
data_path = 'drive/Shareddrives/229 proj/compressed_dataset'

# modify fc layer so output matches our num_classes

num_resnet = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_resnet, num_classes)
model_alexnet.classifier[6] = nn.Linear(4096, num_classes)
model_vgg.classifier[6] = nn.Linear(4096, num_classes)
model_squeezenet.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
model_densenet.classifier = nn.Linear(1024, num_classes)
model_inception.AuxLogits.fc = nn.Linear(768, num_classes)
model_inception.fc = nn.Linear(2048, num_classes)


# print(model)

# Resnet50 takes inputs of dim (224,224,3)
input_size = 224

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #idk what to hardcode these to
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #idk what to hardcode these to
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_path), data_transforms[x]) for x in ['train', 'val']}


# print(image_datasets['train'])
# print(preprocess_resnet)

# print(image_datasets)

# Next step:
# Define a PyTorch data loader for the images
# data_loader = torch.utils.data.DataLoader(
#     ImageFolder(img_dir, transform=transforms.ToTensor()),
#     batch_size=batch_size,
#     shuffle=True
# )

dataloaders_dict = {
    x: torch.utils.data.DataLoader(image_datasets[x], 
                                   batch_size=batch_size, 
                                   shuffle=True, 
                                   num_workers=2
                                  ) 
    for x in ['train', 'val']
}

In [ ]:
# Get all the countries
sorted_countries = sorted([country for country in os.listdir(data_path) if '.' not in country])

countries = {i: (country, len(sorted_countries[i])) for i, country in enumerate(sorted_countries)}
print(countries)

{0: ('Aland', 5), 1: ('Albania', 7), 2: ('American Samoa', 14), 3: ('Andorra', 7), 4: ('Antarctica', 10), 5: ('Argentina', 9), 6: ('Armenia', 7), 7: ('Australia', 9), 8: ('Austria', 7), 9: ('Bangladesh', 10), 10: ('Belarus', 7), 11: ('Belgium', 7), 12: ('Bermuda', 7), 13: ('Bhutan', 6), 14: ('Bolivia', 7), 15: ('Botswana', 8), 16: ('Brazil', 6), 17: ('Bulgaria', 8), 18: ('Cambodia', 8), 19: ('Canada', 6), 20: ('Chile', 5), 21: ('China', 5), 22: ('Colombia', 8), 23: ('Costa Rica', 10), 24: ('Croatia', 7), 25: ('Curacao', 7), 26: ('Czechia', 7), 27: ('Denmark', 7), 28: ('Dominican Republic', 18), 29: ('Ecuador', 7), 30: ('Egypt', 5), 31: ('Estonia', 7), 32: ('Eswatini', 8), 33: ('Faroe Islands', 13), 34: ('Finland', 7), 35: ('France', 6), 36: ('Germany', 7), 37: ('Ghana', 5), 38: ('Gibraltar', 9), 39: ('Greece', 6), 40: ('Greenland', 9), 41: ('Guam', 4), 42: ('Guatemala', 9), 43: ('Hong Kong', 9), 44: ('Hungary', 7), 45: ('Iceland', 7), 46: ('India', 5), 47: ('Indonesia', 9), 48: ('Iraq'

In [ ]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Send the model to GPU / CPU
model_resnet = model_resnet.to(device)

In [ ]:

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_resnet.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_resnet.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_resnet.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Optimize the params in params_to_update using SGD
optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
# model training code 
import numpy as np
def debug_time(identifier,start):
    print(f"part {identifier} at {time.time()-start}")

def train_model(model, dataloaders, criterion, optimizer, num_epochs, is_inception=False):
    since = time.time()
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            start = time.time()
            debug_time(0,start)
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data.
            # start_for = time.time()
            start = time.time()
            temp = dataloaders[phase]
            for inputs, labels in temp:
                print(f"inputs size : {np.shape(inputs)}")
                debug_time(4,start)
                start = time.time()
                debug_time(1,start)
                # end_for = time.time()
                # print(f"{phase} for loop took {end_for - start_for} sec to complete")
                start_for = time.time()
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # print(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                # start_with = time.time()
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)


                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # end_with = time.time()
                # print(f"with thing took {end_with - start_with}")
                debug_time(2,start)
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                end = time.time()
                #print(f"body run time: {end - start} sec")
                debug_time(3,start)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            debug_time(5,start)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            debug_time(6,start)
        print()
      
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train  model
model, hist = train_model(model_resnet, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)



Epoch 0/15
----------
part 0 at 2.6226043701171875e-06
inputs size : torch.Size([32, 3, 224, 224])
part 4 at 5.821446657180786
part 1 at 1.1920928955078125e-06
part 2 at 0.026277780532836914
part 3 at 0.13277459144592285
inputs size : torch.Size([32, 3, 224, 224])
part 4 at 0.13397645950317383
part 1 at 2.86102294921875e-06
part 2 at 0.01974034309387207
part 3 at 0.1030116081237793
inputs size : torch.Size([32, 3, 224, 224])
part 4 at 4.5968921184539795
part 1 at 7.152557373046875e-07
part 2 at 0.02789926528930664
part 3 at 0.13463449478149414
inputs size : torch.Size([32, 3, 224, 224])
part 4 at 1.3201398849487305
part 1 at 7.152557373046875e-07
part 2 at 0.04831099510192871
part 3 at 0.13479948043823242
inputs size : torch.Size([32, 3, 224, 224])
part 4 at 4.095383167266846
part 1 at 9.5367431640625e-07
part 2 at 0.021726369857788086
part 3 at 0.13126325607299805
inputs size : torch.Size([32, 3, 224, 224])
part 4 at 0.37659311294555664
part 1 at 7.152557373046875e-07
part 2 at 0.0194

In [ ]:
######### Baseline Section End ############################################# 


In [ ]:
url = "../input/geolocation-geoguessr-images-50k/compressed_dataset/Reunion/"
!ls
!cat sample_data/README.md


In [ ]:
# chat-GPT filler code:
# Import the necessary libraries
import torch
import torchvision
from torch.utils.data import random_split
from torch.optim import SGD

# Load the pre-trained ResNet model
model = torchvision.models.resnet18(pretrained=True)

# Set the model to evaluation mode
model.eval()

# Load your own data
data = ...

# Split the data into a train set and a test set
train_data, test_data = random_split(data, [90, 10])

# Define the cross entropy loss function
criterion = torch.nn.CrossEntropyLoss()

# Define the SGD optimizer
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
for input, target in train_data:
    # Forward pass
    output = model(input)
    loss = criterion(output, target)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()

    # Update the weights
    optimizer.step()

# Test the model
with torch.no_grad():
    for input, target in test_data:
        # Forward pass
        output = model(input)

        # Compute the loss
        loss = criterion(output, target)

        # Print the loss
        print(loss)
